## Scenario: A Token Issuer maintains a Token Status List at the Status List Server

In [1]:
from helpers import fixtures, utils
from helpers.utils import tc
import requests
import uuid
import jwt

### Configure the base URL of the Status List Server

In [2]:
BASE_URL = utils.get_base_url()
print(f"Using base URL: {BASE_URL}")

Using base URL: http://localhost:8000


### Test the Health Check Endpoint (GET /health)

In [3]:
health_endpoint = f"{BASE_URL}/health"
print(f"Testing GET: {health_endpoint}")

response = requests.get(health_endpoint)
tc.assertEqual(response.status_code, 200, "Health check failed")
tc.assertEqual("OK", response.json().get("status"))

print("Health check successful! Server is running.")

Testing GET: http://localhost:8000/health
Health check successful! Server is running.


### Publish credentials to register as an Issuer (POST /credentials)

In [4]:
issuer_data = fixtures.get_gondwana_digital_pole_issuer()
print("Proceeding with Issuer:", issuer_data.get('label'))

credentials_endpoint = f"{BASE_URL}/credentials"
print(f"Testing POST: {credentials_endpoint}")

payload = {
  "issuer": issuer_data.get('label'),
  "public_key": issuer_data.get('keypair').get('public_key'),
  "alg": issuer_data.get('keypair').get('alg')
}

response = requests.post(credentials_endpoint, json=payload)
tc.assertEqual(response.status_code, 202, "Failed to publish credentials")
tc.assertEqual("Credentials stored successfully", response.text)
print("Credentials published successfully!")

Proceeding with Issuer: gondwana-digital-pole-b4kpn
Testing POST: http://localhost:8000/credentials
Credentials published successfully!


In [5]:
# It should not be possible to replay the request to publish credentials for the same issuer.
response = requests.post(credentials_endpoint, json=payload)
tc.assertEqual(response.status_code, 409)
tc.assertEqual("Credentials already exist for this issuer", response.text)

### Publish token statuses to a status list (POST /statuslists/publish)

In [6]:
status_publish_endpoint = f"{BASE_URL}/statuslists/publish"
print(f"Testing POST: {status_publish_endpoint}")

# Issuers are responsible of the ID of the status list they manage
status_list_id = str(uuid.uuid4())
print("Publishing status list:", status_list_id)

# Create bearer JWT token for authentication
bearer_token = fixtures.create_bearer_jwt_token(issuer_data)

# Publish statuses
response = requests.post(
    status_publish_endpoint,
    headers={"Authorization": f"Bearer {bearer_token}"},
    json={
        "list_id": status_list_id,
        "status": [
            {"index": 1, "status": "VALID"},
            {"index": 8, "status": "INVALID"}
        ]
    }
)

tc.assertEqual(response.status_code, 201, "Failed to publish token statuses")
print("Token statuses published successfully!")

Testing POST: http://localhost:8000/statuslists/publish
Publishing status list: b01aa06f-a499-46ff-af47-f2b4afa4d17b
Token statuses published successfully!


### Update token statuses given a status list (POST /statuslists/{list_id})

In [ ]:
status_update_endpoint = f"{BASE_URL}/statuslists/{status_list_id}"
print(f"Testing PUT: {status_update_endpoint}")

# Create bearer JWT token for authentication
bearer_token = fixtures.create_bearer_jwt_token(issuer_data)

# Publish statuses
response = requests.post(
    status_publish_endpoint,
    headers={"Authorization": f"Bearer {bearer_token}"},
    json={
        "list_id": status_list_id,
        "status": [
            {"index": 1, "status": "VALID"},
            {"index": 8, "status": "INVALID"}
        ]
    }
)

tc.assertEqual(response.status_code, 201, "Failed to publish token statuses")
print("Token statuses published successfully!")

In [7]:
# Unfortunately, there is currently no means to update status lists
# Attempting to publish again under the same status list ID results in a 409 error.

response = requests.post(status_publish_endpoint, json={
    "list_id": status_list_id,
    "bits": 2,
    "updates": [
        {"index": 1, "status": "VALID"},
        {"index": 8, "status": "INVALID"}
    ]
})

tc.assertEqual(response.status_code, 409)

AssertionError: 401 != 409

### A Relying Party retrieves published status lists (GET /statuslists/{status_list_id})

In [ ]:
# application/statuslist+jwt

status_retrieve_endpoint = f"{BASE_URL}/statuslists/{status_list_id}"
print(f"Testing GET: {status_retrieve_endpoint}")

headers = {'Accept': 'application/statuslist+jwt'}
response = requests.get(status_retrieve_endpoint, headers=headers)
tc.assertEqual(response.status_code, 200, "Failed to retrieve status list")

print("Retrieved status list successfully:", end=" ")
jwt_token = response.text
print(jwt_token)

# validate JWT

header = jwt.get_unverified_header(jwt_token)
payload = jwt.decode(jwt_token, options={"verify_signature": False})

tc.assertEqual('statuslist+jwt', header.get('typ'))
tc.assertEqual({'bits': 2, 'lst': 'eNpjYGAEAAAEAAI'}, payload.get('status_list'))

lst = payload.get('status_list').get('lst')
tc.assertEqual(bytes([0x00, 0x00, 0x01]), utils.decode_and_decompress(lst))

In [ ]:
# application/statuslist+cwt

headers = {'Accept': 'application/statuslist+cwt'}
response = requests.get(status_retrieve_endpoint, headers=headers)
tc.assertEqual(response.status_code, 200, "Failed to retrieve status list")
tc.assertTrue(utils.is_valid_cwt(response.content))

print("Retrieved status list successfully")